In [1]:
# Import all necessary packages
import os
from os import sep
#import pubchempy as pcp
import pandas as pd
from collections import defaultdict
import numpy as np
from numpy import nan
from IPython.display import Image
#import tabula

#from tabula import read_pdf
#from tabulate import tabulate

In [2]:
#import geckodriver_autoinstaller
#geckodriver_autoinstaller.install();

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.service import Service
import csv
import requests
import time


In [8]:
from tqdm.auto import tqdm

## Import Data

In [9]:
!pwd

/home/ramin/Desktop/personal/cheminfo/PK_ADME/Code


In [3]:
df = pd.read_csv('/home/ramin/Desktop/Thesis/ripk/ripk_1/5tx5/better_than_cc_p.csv', engine='pyarrow', dtype_backend='pyarrow')

In [4]:
df.head()

,name,deltaG,SMILES
0,pubchem_346,-11.93,Cc1nc(-c2cccc(NC(=O)COc3ccccc3Cc3ccccc3)c2)n[nH]1
1,pubchem_1646,-11.72,O=C(Cn1c(=O)oc2ccccc21)NCc1nncn1CCc1ccccc1
2,pubchem_1713,-11.65,Cc1nc(-c2cccc(NC(=O)CCc3cccc(OC4CCCC4)c3)c2)n[...
3,pubchem_1633,-11.47,O=C(Cn1c(=O)oc2ccccc21)N1CCC(Cc2nc(C3CC3)n[nH]...
4,pubchem_1220,-11.44,O=C(Nc1ccccc1OCc1ccccc1)c1nnnn1Cc1ccccc1


In [48]:
Smiles = df.SMILES.values

## SwissADME
---

In [47]:
defult_atts = ['Canonical SMILES', 'Synthetic Accessibility', 'GI absorption', 'BBB permeant', 'Pgp substrate']

In [58]:
import time
import pandas as pd
import numpy as np
import re

# Split Smiles list into batches of 10
batch_size = 10
num_batches = int(np.ceil(len(Smiles) / batch_size))
batched_smiles = np.array_split(Smiles, num_batches)

# Create list for SwissADME links for each batch
batch_links = []

# Name each elements for code
name = "smiles"
button = "submitButton"
website = "http://www.swissadme.ch"

# Set up Chrome options
options = Options()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--headless")

# Process each batch
for batch_num, smiles_batch in enumerate([Smiles]):
    print(f"Processing batch {batch_num + 1} of {len(batched_smiles)}")
    
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()), options=options
    )
    wait = WebDriverWait(driver, 30)
    
    try:
        # Open SwissADME database
        driver.get(website)
        
        # Locate search box and input smiles
        element1 = wait.until(EC.visibility_of_element_located((By.NAME, name)))
        smiles_search_ADME = driver.find_element(By.NAME, name)
        
        # Input all SMILES from current batch
        for smiles in smiles_batch:
            smiles_search_ADME.send_keys(smiles)
            smiles_search_ADME.send_keys(Keys.RETURN)
        
        # Submit the batch
        element2 = wait.until(EC.presence_of_element_located((By.ID, button)))
        run = driver.find_element(By.ID, button).submit()
        
        # Wait for results to load (wait for mol-cell-1 which indicates results are ready)
        wait.until(EC.presence_of_element_located((By.ID, "mol-cell-1")))
        
        # Get page source and extract temp directory
        page_source = driver.page_source
        temp_dir_match = re.search(r'var temp_directory="([^"]+)"', page_source)
        
        if temp_dir_match:
            temp_directory = temp_dir_match.group(1)
            csv_link = website + '/' + '/'.join([temp_directory.split('/')[i] for i in [-2, -1]]) + '/swissadme.csv'
            batch_links.append(csv_link)
            print(f"Successfully extracted CSV link for batch {batch_num + 1}: {csv_link}")
        else:
            print(f"Could not find temp directory in batch {batch_num + 1}")
            batch_links.append(None)
        
        # Wait to ensure data is processed
        time.sleep(10)
        
    except Exception as e:
        print(f"Error processing batch {batch_num + 1}: {str(e)}")
        batch_links.append(None)
        
    finally:
        driver.close()
        time.sleep(2)  # Brief pause between batches

print("\nAll batches processed. CSV links obtained:")
for i, link in enumerate(batch_links):
    print(f"Batch {i+1}: {link}")

all_data = []
for i, link in enumerate(batch_links):
    if link:
        try:
            batch_data = pd.read_csv(link)
            all_data.append(batch_data[defult_atts])
            print(f"Successfully read data from batch {i+1}")
        except Exception as e:
            print(f"Error reading CSV from batch {i+1}: {str(e)}")

# Combine all batches into one DataFrame
if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    del all_data
    print(f"\nFinal DataFrame shape: {final_df.shape}")
else:
    print("No data was successfully collected")

Processing batch 1 of 91


MaxRetryError: HTTPConnectionPool(host='localhost', port=40185): Max retries exceeded with url: /session/36934d6b495b33c9ef1e1cc0cf0094dd/window (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7681c1a9f140>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [40]:
website + '/' + '/'.join([temp_directory.split('/')[i] for i in [-2, -1]]) + '/swissadme.csv'

'http://www.swissadme.ch/results/341228269/swissadme.csv'